In [2]:
library(tidyverse)

In [3]:
root_path     <- here::here()
data_path     <- paste0(root_path, "/01_data")
analysis_path <- paste0(root_path, "/02_analysis")

In [4]:
setwd(data_path)

candidate <- readRDS("clean/candidate_clean.rds")

transplant <- readRDS("clean/transplant_clean.rds")

state_code_mapping <- read.csv("raw/00_misc/state_code_mapping_20221022.csv")

setwd(analysis_path)

deathTransplantListingRates.orig <- read.csv("output/tables/table2-per_capita_rates_by_race_state.csv")

---

### Re-calculating transplants + listings counts + rates

Use candidate's permanent state of residence (`CAN_PERM_STATE`) as opposed to state of hospital (since candidate's often travel).

In [5]:
# calculate transplant counts (by race, year, state)
transplantCounts.StateYearRace <- transplant %>%
  filter(REC_TX_DT_YEAR %in% 2018:2021) %>%
  filter(!grepl("PR|VI|GU|NA|ZZ", CAN_PERM_STATE_CLEAN)) %>%
  count(STATE_CODE = CAN_PERM_STATE_CLEAN, YEAR = as.character(REC_TX_DT_YEAR), RACE = CAN_RACE_SRTR_CLEAN,
        name = "TRANSPLANTS")

# aggregate and stack transplantCounts at multiple agg levels (to match cdcWonder)
transplantCounts <- bind_rows(transplantCounts.StateYearRace,
                              transplantCounts.StateYearRace %>% # StateYear
                                count(STATE_CODE, YEAR, RACE = "All Races",
                                      wt = TRANSPLANTS, name = "TRANSPLANTS"),
                              transplantCounts.StateYearRace %>% # StateRace
                                count(STATE_CODE, YEAR = "2018 - 2021", RACE,
                                      wt = TRANSPLANTS, name = "TRANSPLANTS"),
                              transplantCounts.StateYearRace %>% # YearRace
                                count(STATE_CODE = "National", YEAR, RACE,
                                      wt = TRANSPLANTS, name = "TRANSPLANTS"),
                              transplantCounts.StateYearRace %>% # Year
                                count(STATE_CODE = "National", YEAR, RACE = "All Races",
                                      wt = TRANSPLANTS, name = "TRANSPLANTS"),
                              transplantCounts.StateYearRace %>% # Race
                                count(STATE_CODE = "National", YEAR = "2018 - 2021", RACE,
                                      wt = TRANSPLANTS, name = "TRANSPLANTS"),
                              transplantCounts.StateYearRace %>% # Total
                                count(STATE_CODE = "National", YEAR = "2018 - 2021", RACE = "All Races",
                                      wt = TRANSPLANTS, name = "TRANSPLANTS")) %>%
  left_join(state_code_mapping %>% select(-STATE_FIPS, -STATE_REGION), by = "STATE_CODE") %>%
  mutate(STATE_NAME = if_else(is.na(STATE_NAME), STATE_CODE, STATE_NAME))

transplantCounts %>% head()

STATE_CODE,YEAR,RACE,TRANSPLANTS,STATE_NAME
<chr>,<chr>,<chr>,<int>,<chr>
AK,2018,American Indian or Alaska Native,2,Alaska
AK,2018,White,2,Alaska
AK,2019,Asian,1,Alaska
AK,2019,White,11,Alaska
AK,2020,American Indian or Alaska Native,2,Alaska
AK,2020,Asian,1,Alaska


In [6]:
# calculate waitlist listing totals (by race, year, state)
waitlistListings.StateYearRace <- candidate %>% 
  filter(CAN_LISTING_DT_YEAR %in% 2018:2021) %>%
  filter(!grepl("PR|VI|GU|NA|ZZ", CAN_PERM_STATE_CLEAN)) %>%
  count(STATE_CODE = CAN_PERM_STATE_CLEAN, YEAR = as.character(CAN_LISTING_DT_YEAR), RACE = CAN_RACE_SRTR_CLEAN,
        name = "WAITLIST_LISTINGS")

# aggregate and stack waitlistListings at multiple agg levels (to match cdcWonder)
waitlistListings <- bind_rows(waitlistListings.StateYearRace,
                              waitlistListings.StateYearRace %>% # StateYear
                                count(STATE_CODE, YEAR, RACE = "All Races",
                                      wt = WAITLIST_LISTINGS, name = "WAITLIST_LISTINGS"),
                              waitlistListings.StateYearRace %>% # StateRace
                                count(STATE_CODE, YEAR = "2018 - 2021", RACE,
                                      wt = WAITLIST_LISTINGS, name = "WAITLIST_LISTINGS"),
                              waitlistListings.StateYearRace %>% # YearRace
                                count(STATE_CODE = "National", YEAR, RACE,
                                      wt = WAITLIST_LISTINGS, name = "WAITLIST_LISTINGS"),
                              waitlistListings.StateYearRace %>% # Year
                                count(STATE_CODE = "National", YEAR, RACE = "All Races",
                                      wt = WAITLIST_LISTINGS, name = "WAITLIST_LISTINGS"),
                              waitlistListings.StateYearRace %>% # Race
                                count(STATE_CODE = "National", YEAR = "2018 - 2021", RACE,
                                      wt = WAITLIST_LISTINGS, name = "WAITLIST_LISTINGS"),
                              waitlistListings.StateYearRace %>% # Total
                                count(STATE_CODE = "National", YEAR = "2018 - 2021", RACE = "All Races",
                                      wt = WAITLIST_LISTINGS, name = "WAITLIST_LISTINGS")) %>%
  left_join(state_code_mapping %>% select(-STATE_FIPS, -STATE_REGION), by = "STATE_CODE") %>%
  mutate(STATE_NAME = if_else(is.na(STATE_NAME), STATE_CODE, STATE_NAME))

waitlistListings %>% head()

STATE_CODE,YEAR,RACE,WAITLIST_LISTINGS,STATE_NAME
<chr>,<chr>,<chr>,<int>,<chr>
AK,2018,American Indian or Alaska Native,2,Alaska
AK,2018,Black or African American,1,Alaska
AK,2018,White,7,Alaska
AK,2019,American Indian or Alaska Native,2,Alaska
AK,2019,Asian,1,Alaska
AK,2019,Black or African American,1,Alaska


In [7]:
deathTransplantListingRates <- deathTransplantListingRates.orig %>%
  select(STATE_NAME, YEAR, RACE, DEATHS, POPULATION, DEATH_RATE, TRANSPLANTS) %>%
  select(-TRANSPLANTS) %>% 
  full_join(transplantCounts %>% select(-STATE_CODE),
            by = c("STATE_NAME", "YEAR", "RACE")) %>%
  full_join(waitlistListings %>% select(-STATE_CODE),
            by = c("STATE_NAME", "YEAR", "RACE")) %>%
  mutate(across(matches("TRANSPLANTS|WAITLIST_LISTINGS"), replace_na, 0)) %>%
  mutate(TRANSPLANT_RATE = TRANSPLANTS / POPULATION * 1e5, .after = TRANSPLANTS) %>%
  mutate(LISTING_RATE = WAITLIST_LISTINGS / POPULATION * 1e5, .after = WAITLIST_LISTINGS) %>%
  mutate(LISTING_DEATH_RATIO = WAITLIST_LISTINGS / DEATHS,
         TRANSPLANT_DEATH_RATIO = TRANSPLANTS / DEATHS,
         TRANSPLANT_LISTING_RATIO = TRANSPLANTS / WAITLIST_LISTINGS)

deathTransplantListingRates %>% head()

,STATE_NAME,YEAR,RACE,DEATHS,POPULATION,DEATH_RATE,TRANSPLANTS,TRANSPLANT_RATE,WAITLIST_LISTINGS,LISTING_RATE,LISTING_DEATH_RATIO,TRANSPLANT_DEATH_RATIO,TRANSPLANT_LISTING_RATIO
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,National,2018,All Races,42838,327167434,13.093601,7622,2.3296940,12331,3.7690182,0.2878519,0.17792614,0.6181169
2,National,2018,American Indian or Alaska Native,1149,4147521,27.703296,75,1.8083091,134,3.2308456,0.1166232,0.06527415,0.5597015
3,National,2018,Asian,671,19330600,3.471180,331,1.7123111,583,3.0159436,0.8688525,0.49329359,0.5677530
4,National,2018,Black or African American,3314,43804319,7.565464,623,1.4222342,916,2.0911180,0.2764031,0.18799034,0.6801310
5,National,2018,More than one race,305,8946480,3.409162,48,0.5365239,80,0.8942064,0.2622951,0.15737705,0.6000000
6,National,2018,Native Hawaiian or Other Pacific Islander,46,799418,5.754186,15,1.8763651,25,3.1272751,0.5434783,0.32608696,0.6000000


In [8]:
# calculate rate differentials across race (i.e., difference between White and non-White)
rateDifferentialsByState <- deathTransplantListingRates %>%
  filter(YEAR == "2018 - 2021", RACE != "All Races") %>%
  filter(!is.na(DEATHS), TRANSPLANTS > 0) %>%
  select(STATE_NAME, RACE, matches("RATE|RATIO")) %>%
  pivot_longer(where(is.numeric), names_to = "METRIC", values_to = "VALUE") %>%
  left_join(x = filter(., RACE != "White"),
            y = filter(., RACE == "White"),
            by = c("STATE_NAME", "METRIC"), suffix = c(".POC", ".WHITE")) %>%
  select(-RACE.WHITE) %>%
  mutate(DIFFERENTIAL = VALUE.WHITE / VALUE.POC)

rateDifferentialsByState %>% head()

STATE_NAME,RACE.POC,METRIC,VALUE.POC,VALUE.WHITE,DIFFERENTIAL
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
National,American Indian or Alaska Native,DEATH_RATE,34.34073849,16.9103423,0.492428
National,American Indian or Alaska Native,TRANSPLANT_RATE,2.03290553,2.8352688,1.394688
National,American Indian or Alaska Native,LISTING_RATE,3.16164087,4.3670070,1.381247
National,American Indian or Alaska Native,LISTING_DEATH_RATIO,0.09206677,0.2582447,2.804972
National,American Indian or Alaska Native,TRANSPLANT_DEATH_RATIO,0.05919807,0.1676648,2.832267
National,American Indian or Alaska Native,TRANSPLANT_LISTING_RATIO,0.64299065,0.6492476,1.009731


---

### In-story data

> "Native Americans are far less likely than other racial groups to gain a spot on the national liver transplant list, despite having the highest rate of death from liver disease, according to an analysis of four years of transplant data by the Markup and The Washington Post."

American Indians and Alaska Natives have a death rate of 34.3 ESLD deaths per 100K people from 2018 - 2021, more than double that of White people (16.9 deaths per 100K) and the highest among all racial groups.

In [9]:
deathTransplantListingRates %>%
    filter(STATE_NAME == "National",
           YEAR == "2018 - 2021") %>%
    select(STATE_NAME, YEAR, RACE, DEATHS, POPULATION, DEATH_RATE) %>%
    arrange(-DEATH_RATE)

STATE_NAME,YEAR,RACE,DEATHS,POPULATION,DEATH_RATE
<chr>,<chr>,<chr>,<int>,<int>,<dbl>
National,2018 - 2021,American Indian or Alaska Native,5811,16921593,34.340738
National,2018 - 2021,White,169320,1001280734,16.910342
National,2018 - 2021,All Races,195423,1314375203,14.868129
National,2018 - 2021,Black or African American,15541,176941629,8.783122
National,2018 - 2021,Native Hawaiian or Other Pacific Islander,193,3293741,5.859599
National,2018 - 2021,Asian,3142,78860018,3.984275
National,2018 - 2021,More than one race,1416,37077488,3.819029


> "Compared to their total number of deaths from liver disease, White people gain a spot on the transplant list almost three times more often than Native Americans, the data shows."

The ratio of waitlistings-to-deaths for White people nationally from 2018 - 2021 was 0.258 listings per death. For Native people, it was 0.092 listings per death, which indicates a differential of 2.805x higher for White people, or **180.5% higher**.

In [10]:
rateDifferentialsByState %>% 
    filter(STATE_NAME == "National",
           METRIC == "LISTING_DEATH_RATIO",
           grepl("American Indian", RACE.POC))

STATE_NAME,RACE.POC,METRIC,VALUE.POC,VALUE.WHITE,DIFFERENTIAL
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
National,American Indian or Alaska Native,LISTING_DEATH_RATIO,0.09206677,0.2582447,2.804972


> "Had transplant rates been equal, nearly 1,000 additional Native people would have received liver transplants between 2018 and 2021."

The national average of Listing-Death Ratio is 0.258 and 5,811 Indigenous people died from ESLD from 2018 - 2021. If Indigenous people had a listing-death ratio comparable to the national average (0.258 vs. 0.092), this would estimate that approximately 1,498 Indigenous people should have been listed, as compared to the 535 that were. This differential (approximately **963 people**) would have recieved transplants if all things equal.

In [11]:
deathTransplantListingRates %>%
    filter(STATE_NAME == "National",
           YEAR == "2018 - 2021",
           RACE %in% c("American Indian or Alaska Native", "All Races")) %>%
    arrange(LISTING_DEATH_RATIO) %>%
    select(STATE_NAME, YEAR, RACE, DEATHS, WAITLIST_LISTINGS, LISTING_DEATH_RATIO)

STATE_NAME,YEAR,RACE,DEATHS,WAITLIST_LISTINGS,LISTING_DEATH_RATIO
<chr>,<chr>,<chr>,<int>,<int>,<dbl>
National,2018 - 2021,American Indian or Alaska Native,5811,535,0.09206677
National,2018 - 2021,All Races,195423,50381,0.25780486


In [12]:
(5811 * 0.25780486) - 535

[1] 963.104

> "Native Americans who do win a spot on the list advance to surgery at about the same rate as White people, showing that front-end access is a primary driver of disparity."

When adjusting the number of transplants relative to the number of people added to the transplant waitlist, roughly equivalent numbers of White and Native American people receive transplants. For every 100 White people added to the waitlist, 64.9 people receive transplants. For every 100 Native American people, 64.3 people receive transplants.

In [13]:
rateDifferentialsByState %>% 
    filter(STATE_NAME == "National",
           METRIC == "TRANSPLANT_LISTING_RATIO",
           grepl("American Indian", RACE.POC))

STATE_NAME,RACE.POC,METRIC,VALUE.POC,VALUE.WHITE,DIFFERENTIAL
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
National,American Indian or Alaska Native,TRANSPLANT_LISTING_RATIO,0.6429907,0.6492476,1.009731


> "Among other racial groups, the liver transplant acceptance rate for Black people is slightly lower than for White patients nationally, while Asian Americans have the highest rate of acceptance to the transplant wait list by far."

> "For every 100 Asian people who died of liver disease, approximately 68 patients were accepted for transplant from 2018 to 2021. Among White people, 26 patients were accepted; among Black people, it was 23."

> "Among Indigenous people, just nine patients were accepted for transplant in that period for every 100 people who died of liver disease."

The listing-to-death ratio for Asian patients is 0.681, indicating that for every 100 Asian people that die of liver disease, **68.1 Asian people are listed for transplant**. See table below for comparable rates for other races.

In [14]:
deathTransplantListingRates %>%
    filter(STATE_NAME == "National",
           YEAR == "2018 - 2021",
           RACE %in% c("Asian", "White", "Black or African American", "American Indian or Alaska Native")) %>%
    select(STATE_NAME, YEAR, RACE, DEATHS, WAITLIST_LISTINGS, LISTING_DEATH_RATIO) %>%
    arrange(-LISTING_DEATH_RATIO)

STATE_NAME,YEAR,RACE,DEATHS,WAITLIST_LISTINGS,LISTING_DEATH_RATIO
<chr>,<chr>,<chr>,<int>,<int>,<dbl>
National,2018 - 2021,Asian,3142,2139,0.68077658
National,2018 - 2021,White,169320,43726,0.25824474
National,2018 - 2021,Black or African American,15541,3577,0.23016537
National,2018 - 2021,American Indian or Alaska Native,5811,535,0.09206677


> "[Liver disease] is the second leading cause of death among Indigenous men ages 35 to 44."

See [CDC Wonder Data](https://wonder.cdc.gov/controller/saved/D158/D358F404). 

> "In D.C. and Arkansas, for example, White people were twice as likely as Black people to be added to the liver transplant list when compared to their rates of death from liver disease."

In Arkansas, the listing-death ratio is 0.091 for Black patients and 0.181 for White patients, indicating **White patients are listed 99.2% more (double) than Black patients**, assuming comparable numbers of deaths. The differential is comparable in Washington, D.C.

In [15]:
rateDifferentialsByState %>% 
    filter(METRIC == "LISTING_DEATH_RATIO",
           grepl("Black", RACE.POC),
           STATE_NAME %in% c("District of Columbia", "Arkansas"))

STATE_NAME,RACE.POC,METRIC,VALUE.POC,VALUE.WHITE,DIFFERENTIAL
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Arkansas,Black or African American,LISTING_DEATH_RATIO,0.09090909,0.1811790,1.992969
District of Columbia,Black or African American,LISTING_DEATH_RATIO,0.32386364,0.6470588,1.997936


> "An estimated 430 additional Black people would have been accepted for liver transplants from 2018 to 2021 had national transplant rates been equal."

The national average of Listing-Death Ratio is 0.258 and 15,541 Black people died from ESLD from 2018 - 2021. So this would estimate that approximately 4,007 Black people should have been listed, as compared to the 3,577 that were. This differential (approximately **430 people**) would have recieved transplants if all things equal.

In [16]:
deathTransplantListingRates %>%
    filter(STATE_NAME == "National",
           YEAR == "2018 - 2021",
           RACE %in% c("Black or African American", "All Races")) %>%
    arrange(TRANSPLANT_LISTING_RATIO) %>%
    select(STATE_NAME, YEAR, RACE, DEATHS, WAITLIST_LISTINGS, LISTING_DEATH_RATIO)

STATE_NAME,YEAR,RACE,DEATHS,WAITLIST_LISTINGS,LISTING_DEATH_RATIO
<chr>,<chr>,<chr>,<int>,<int>,<dbl>
National,2018 - 2021,All Races,195423,50381,0.2578049
National,2018 - 2021,Black or African American,15541,3577,0.2301654


In [17]:
(15541 * 0.25780486) - 3577

[1] 429.5453

---

### Data exports

In [18]:
deathTransplantListingRates %>%
    filter(STATE_NAME == "National",
           YEAR == "2018 - 2021",
           RACE %in% c("Asian", "White", "Black or African American", "American Indian or Alaska Native")) %>%
    select(STATE_NAME, YEAR, RACE, DEATHS, WAITLIST_LISTINGS, LISTING_DEATH_RATIO) %>%
    arrange(-LISTING_DEATH_RATIO) %>%
    write.csv(paste0(analysis_path, "/output/tables/table9-listing_death_ratio_by_race.csv"),
              row.names = F)

In [19]:
deathTransplantListingRates %>%
    filter(STATE_NAME %in% c("Alabama", "Arkansas", "Georgia", "Iowa", "South Carolina", "Mississippi"),
           YEAR == "2018 - 2021") %>%
    select(STATE_NAME, YEAR, RACE, DEATHS) %>%
    group_by(STATE_NAME, YEAR) %>%
    mutate(DEATHS = replace_na(DEATHS, replace = 0),
           STATE_DEATH_SHARE = DEATHS / sum(DEATHS, na.rm = T)) %>%
    write.csv(paste0(analysis_path, "/output/tables/table9-deaths_by_race_by_state.csv"),
              row.names = F)